# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [1]:
import itertools
from collections import Counter
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [2]:
import numpy as np
from functools import reduce
def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    # YOUR CODE HERE
    return list(reduce(lambda x,y: set(x).union(y), corpus))

In [3]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [4]:

def initalize_translation_prob(corpus1, corpus2):
    # YOUR CODE HERE
    uniform = {}
    for wTarget in get_vocabulary(corpus1):
        for wSource in get_vocabulary(corpus2):
            uniform[(wSource, wTarget)] = 1
    total = sum(uniform.values(), 0.0)
    for englishW in get_vocabulary(corpus1):
        summ = 0.0
        for spanishW in  get_vocabulary(corpus2):
            summ += uniform[(spanishW, englishW)]
        for spanishW in  get_vocabulary(corpus2):
            uniform[(spanishW, englishW)] /= summ
    return uniform
translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [5]:
translation_prob

{('verde', 'the'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333,
 ('la', 'the'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333,
 ('casa', 'green'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333,
 ('la', 'house'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [6]:
def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach 
    (reprezentacja dowolna, nie weryfikowana przez sprawdzarkę)
    """
    
    sentencesExpectations = []
    for sentenceSpanish, sentenceEnglish in zip(corpora2, corpora1):
        a = np.zeros((len(sentenceSpanish), len(sentenceEnglish)))
        for indS,wordS in enumerate(sentenceSpanish):
            for indE,wordE in enumerate(sentenceEnglish):
                a[indS,indE] = translation_prob[(wordS, wordE)]
        a/= np.sum(a, axis=0)
        sentencesExpectations.append(a)
    return sentencesExpectations
    
assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [7]:
assigment_expected_values

[array([[0.5, 0.5],
        [0.5, 0.5]]),
 array([[0.5, 0.5],
        [0.5, 0.5]]),
 array([[0.33333333, 0.33333333, 0.33333333],
        [0.33333333, 0.33333333, 0.33333333],
        [0.33333333, 0.33333333, 0.33333333]])]

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [8]:
def calculate_maximization(corpora1, corpora2, assigment_expected_values):
    prob = Counter()
    for sentenceExpectation, sentenceSpanish, sentenceEnglish in zip(assigment_expected_values, corpora2, corpora1):
        
        for indS,wordS in enumerate(sentenceSpanish):
            
            for indE, wordE in enumerate(sentenceEnglish):
                
                prob[(wordS, wordE)] += sentenceExpectation[indS, indE]
    
    for englishW in get_vocabulary(corpora1):
        summ = 0.0
        for spanishW in  get_vocabulary(corpora2):
            summ += prob[(spanishW, englishW)]
        for spanishW in  get_vocabulary(corpora2):
            prob[(spanishW, englishW)] /= summ
    return prob
translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [9]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [10]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('casa', 'house'), 0.5584045584045584), (('verde', 'house'), 0.22079772079772075), (('la', 'house'), 0.22079772079772075)]
---
[(('casa', 'house'), 0.6638923177619095), (('verde', 'house'), 0.16805384111904523), (('la', 'house'), 0.16805384111904523)]
---
[(('casa', 'house'), 0.7532968646774506), (('verde', 'house'), 0.12335156766127475), (('la', 'house'), 0.12335156766127475)]
---
[(('casa', 'house'), 0.8239601969358897), (('verde', 'house'), 0.08801990153205519), (('la', 'house'), 0.08801990153205519)]
---
[(('casa', 'house'), 0.8769766282684491), (('verde', 'house'), 0.06151168586577549), (('la', 'house'), 0.06151168586577549)]
---
[(('casa', 'house'), 0.915296630096382), (('verde', 'house'), 0.042351684951809), (('la', 'house'), 0.042351684951809)]
---
[(('casa', 'house'), 0.9422824270785402), (('verde', 'house'), 0.02885878646072994), (('la', 'house'), 0.02885878646072994)]
---
[(('casa', 'house'), 0.9609498992371662), (('verde', 'house'), 0.019525050381416928), (('la', 'house')

Wywołaj algorytm EM na poniższym korpusie.

In [15]:
## Zakładam, że tłumaczymy z EN -> PL we wszystkich przykładach i z ESP -> EN, ponieważ tak sugerują kolejne polecenia
# -> mamy dodać NULL do PL, a to w zdaniach, na które tłumaczymy dodajemy NULL wg. wykładu 
# (rozumiem to tak , że język źródłowy emituje tokeny docelowego i NULL służy żeby można było nie wyemitować nic w docelowym ze źródłowego)

english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [17]:
# YOUR CODE HERE
translation_prob = initalize_translation_prob(polish, english2)
for i in range(10):
    assigment_expected_values = calculate_expectation(polish, english2, translation_prob)
    translation_prob = calculate_maximization(polish, english2, assigment_expected_values)

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [18]:
translation_prob

Counter({('the', 'pies'): 0.5,
         ('dog', 'pies'): 0.5,
         ('the', 'dom'): 0.49983723958333337,
         ('house', 'dom'): 0.49983723958333337,
         ('the', 'zielony'): 0.3333333333333333,
         ('green', 'zielony'): 0.3333333333333333,
         ('green', 'dom'): 0.00032552083333333337,
         ('house', 'zielony'): 0.3333333333333333,
         ('dog', 'dom'): 0.0,
         ('house', 'pies'): 0.0,
         ('green', 'pies'): 0.0,
         ('dog', 'zielony'): 0.0})

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [14]:
# YOUR CODE HERE
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["NULL", "pies"], ["NULL", "dom"], ["NULL","zielony", "dom"]]

translation_prob = initalize_translation_prob(polish, english2)
for i in range(10):
    assigment_expected_values = calculate_expectation(polish, english2, translation_prob)
    translation_prob = calculate_maximization(polish, english2, assigment_expected_values)

In [32]:
translation_prob

Counter({('the', 'NULL'): 0.9884935162981091,
         ('the', 'pies'): 0.5,
         ('dog', 'NULL'): 3.5095620537396894e-05,
         ('dog', 'pies'): 0.5,
         ('the', 'dom'): 0.49983723958333337,
         ('house', 'NULL'): 0.01146338484277804,
         ('house', 'dom'): 0.49983723958333337,
         ('the', 'zielony'): 0.3333333333333333,
         ('green', 'NULL'): 8.003238575501878e-06,
         ('green', 'zielony'): 0.3333333333333333,
         ('green', 'dom'): 0.00032552083333333337,
         ('house', 'zielony'): 0.3333333333333333,
         ('dog', 'dom'): 0.0,
         ('dog', 'zielony'): 0.0,
         ('green', 'pies'): 0.0,
         ('house', 'pies'): 0.0})

Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

Dzieje się tak, ponieważ "casa" i jego poprawne tłumaczenie "house" występuje w każdym zdaniu w korpusie. "NULL" wstawiamy zawsze w zdaniu (na początku) w języku, na który tłumaczymy. Model opiera się na zliczaniu współwystępowania w zdaniach równoległych, więc model nie potrafi rozróżnić wpływu "casa" na "NULL"  i "house". Widać to w przykładzie poniżej. 

Wystarczyłaby jedna para zdań więcej, która naturalnie posiadałoby "NULL" w języku angielskim, a nie posiadała "casa" (a co za tym idzie również tłumaczenia "house") w języku źródłowym. Dzięki temu model mógłby dostrzec fakt, że brak obecności "casa" wpływa na brak "house" w tłumaczeniu, a nie na brak "NULL". Widać to na drugim przykładzie niżej.

Problem może być większy dla słów, które można przetłumaczyć na kilka różnych wyrazów, wtedy "NULL" łatwiej przejmie większość prawdopodobieństwa! W praktyce korpusy równoległe prawdopodobnie nie posiadają w prawie każdym zdaniu tego samego słowa, więc ten problem raczej nie ma krytycznego znaczenia. 

In [37]:
english = [["NULL","green","house"], ["NULL","the","house"], ["NULL","the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

translation_prob = initalize_translation_prob(english, spanish)
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [38]:
translation_prob

Counter({('casa', 'NULL'): 0.9737073866187812,
         ('casa', 'green'): 0.49983723958333337,
         ('casa', 'house'): 0.9737073866187812,
         ('verde', 'NULL'): 0.013146306690609386,
         ('verde', 'green'): 0.49983723958333337,
         ('verde', 'house'): 0.013146306690609386,
         ('la', 'NULL'): 0.013146306690609386,
         ('la', 'the'): 0.49983723958333337,
         ('la', 'house'): 0.013146306690609386,
         ('casa', 'the'): 0.49983723958333337,
         ('la', 'green'): 0.00032552083333333337,
         ('verde', 'the'): 0.00032552083333333337})

In [35]:
english = [["NULL","green","house"], ["NULL","the","house"], ["NULL","the", "green", "house"], ["NULL", "green"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"], ["verde"]]

translation_prob = initalize_translation_prob(english, spanish)
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [36]:
translation_prob

Counter({('casa', 'NULL'): 0.4722873474087891,
         ('casa', 'green'): 0.007898147029388158,
         ('casa', 'house'): 0.9737073866187812,
         ('verde', 'NULL'): 0.5091188186400821,
         ('verde', 'green'): 0.9920962079413423,
         ('verde', 'house'): 0.013146306690609386,
         ('la', 'NULL'): 0.01859383395112877,
         ('la', 'the'): 0.49983723958333337,
         ('la', 'house'): 0.013146306690609386,
         ('casa', 'the'): 0.49983723958333337,
         ('la', 'green'): 5.645029269476762e-06,
         ('verde', 'the'): 0.00032552083333333337})